In [1]:
from sklearn import datasets, tree, metrics, ensemble
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import numpy as np
import scipy
import time
from sklearn.model_selection import train_test_split
import pandas as pd
import statistics
import einops

In [2]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [3]:
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # dBias = output_error

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [4]:
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [5]:
def tanh(x):
    return np.tanh(x);

def tanh_prime(x):
    return 1-np.tanh(x)**2;

In [6]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

In [7]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [8]:
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# network
net = Network()
net.add(FCLayer(2, 3))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(3, 1))
net.add(ActivationLayer(tanh, tanh_prime))

# train
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=1000, learning_rate=0.1)

# test
out = net.predict(x_train)
print(out)

epoch 1/1000   error=0.551375
epoch 2/1000   error=0.336744
epoch 3/1000   error=0.305169
epoch 4/1000   error=0.297537
epoch 5/1000   error=0.294875
epoch 6/1000   error=0.293662
epoch 7/1000   error=0.292978
epoch 8/1000   error=0.292522
epoch 9/1000   error=0.292177
epoch 10/1000   error=0.291891
epoch 11/1000   error=0.291641
epoch 12/1000   error=0.291414
epoch 13/1000   error=0.291203
epoch 14/1000   error=0.291005
epoch 15/1000   error=0.290817
epoch 16/1000   error=0.290638
epoch 17/1000   error=0.290466
epoch 18/1000   error=0.290302
epoch 19/1000   error=0.290145
epoch 20/1000   error=0.289994
epoch 21/1000   error=0.289848
epoch 22/1000   error=0.289708
epoch 23/1000   error=0.289573
epoch 24/1000   error=0.289442
epoch 25/1000   error=0.289316
epoch 26/1000   error=0.289194
epoch 27/1000   error=0.289076
epoch 28/1000   error=0.288961
epoch 29/1000   error=0.288850
epoch 30/1000   error=0.288742
epoch 31/1000   error=0.288638
epoch 32/1000   error=0.288536
epoch 33/1000   e

epoch 576/1000   error=0.000906
epoch 577/1000   error=0.000902
epoch 578/1000   error=0.000897
epoch 579/1000   error=0.000893
epoch 580/1000   error=0.000889
epoch 581/1000   error=0.000884
epoch 582/1000   error=0.000880
epoch 583/1000   error=0.000876
epoch 584/1000   error=0.000872
epoch 585/1000   error=0.000868
epoch 586/1000   error=0.000864
epoch 587/1000   error=0.000860
epoch 588/1000   error=0.000856
epoch 589/1000   error=0.000852
epoch 590/1000   error=0.000848
epoch 591/1000   error=0.000844
epoch 592/1000   error=0.000840
epoch 593/1000   error=0.000836
epoch 594/1000   error=0.000832
epoch 595/1000   error=0.000829
epoch 596/1000   error=0.000825
epoch 597/1000   error=0.000821
epoch 598/1000   error=0.000817
epoch 599/1000   error=0.000814
epoch 600/1000   error=0.000810
epoch 601/1000   error=0.000807
epoch 602/1000   error=0.000803
epoch 603/1000   error=0.000800
epoch 604/1000   error=0.000796
epoch 605/1000   error=0.000793
epoch 606/1000   error=0.000789
epoch 60

In [9]:
from keras.datasets import mnist
from keras.utils import np_utils


(x_train, y_train), (x_test, y_test) = mnist.load_data()

# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = np_utils.to_categorical(y_train)

# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = np_utils.to_categorical(y_test)

# Network
net = Network()
net.add(FCLayer(28*28, 100))                # input_shape=(1, 28*28)    ;   output_shape=(1, 100)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(100, 50))                   # input_shape=(1, 100)      ;   output_shape=(1, 50)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(50, 10))                    # input_shape=(1, 50)       ;   output_shape=(1, 10)
net.add(ActivationLayer(tanh, tanh_prime))

# train on 1000 samples
# as we didn't implemented mini-batch GD, training will be pretty slow if we update at each iteration on 60000 samples...
net.use(mse, mse_prime)
net.fit(x_train[0:1000], y_train[0:1000], epochs=35, learning_rate=0.1)

# test on 3 samples
out = net.predict(x_test[0:3])
print("\n")
print("predicted values : ")
print(out, end="\n")
print("true values : ")
print(y_test[0:3])

epoch 1/35   error=0.247079
epoch 2/35   error=0.084119
epoch 3/35   error=0.065393
epoch 4/35   error=0.055877
epoch 5/35   error=0.049075
epoch 6/35   error=0.043731
epoch 7/35   error=0.039254
epoch 8/35   error=0.035446
epoch 9/35   error=0.032300
epoch 10/35   error=0.029670
epoch 11/35   error=0.027377
epoch 12/35   error=0.025340
epoch 13/35   error=0.023572
epoch 14/35   error=0.022008
epoch 15/35   error=0.020616
epoch 16/35   error=0.019337
epoch 17/35   error=0.018155
epoch 18/35   error=0.017083
epoch 19/35   error=0.016161
epoch 20/35   error=0.015305
epoch 21/35   error=0.014494
epoch 22/35   error=0.013710
epoch 23/35   error=0.013003
epoch 24/35   error=0.012373
epoch 25/35   error=0.011813
epoch 26/35   error=0.011310
epoch 27/35   error=0.010847
epoch 28/35   error=0.010390
epoch 29/35   error=0.009979
epoch 30/35   error=0.009612
epoch 31/35   error=0.009273
epoch 32/35   error=0.008959
epoch 33/35   error=0.008655
epoch 34/35   error=0.008335
epoch 35/35   error=0.0